### Move to the full Nicholls-Turton entrainment parameterization, following Gesso et al. 2014

Read in the model result for the simple radiative entrainment closure of de Roode lecture 2 and use those
values to calculate the NT entrainment rate

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
from a500_utils.helpersII import make_tuple
from thermlib import thermfuncsII as tf
from thermlib import thermconst as tc
from importlib import reload
reload(tf)
import pandas as pd
import numpy as np

with pd.HDFStore('dumpvapor.h5') as f:
    df_result=f['profile']
out=df_result.plot('time','h')



### turn the last timestep values into a named tuple

In [ ]:
out=df_result.iloc[-1]
print(out)
steady_state=make_tuple(out.to_dict())
steady_state



### plot the buoyancy flux profile

In [ ]:
def flux_prof(z,coeffs):
    Fout=coeffs.Fsfc + (coeffs.Finv - coeffs.Fsfc)*(z/coeffs.zinv)
    return Fout

zvals=np.linspace(0,steady_state.h,30)
coeffs_thetal=make_tuple(dict(Fsfc=steady_state.T_flux_0,Finv=steady_state.entflux_theta,
                       zinv=steady_state.h))
coeffs_qt=make_tuple(dict(Fsfc=steady_state.q_flux_0,Finv=steady_state.entflux_qv,
                       zinv=steady_state.h))
flux_thetal=flux_prof(zvals,coeffs_thetal)
flux_qt=flux_prof(zvals,coeffs_qt)
df_flux=pd.DataFrame.from_records(zip(zvals,flux_thetal,flux_qt),columns=['z','flux_thetal','flux_qt'])

plt.close('all')
fig,ax = plt.subplots(1,2,figsize=(12,10))
df_flux.plot('flux_thetal','z',ax=ax[0])
out=ax[1].plot(flux_qt*2.5e6,zvals)
ax[0].set(title=r'$q_t\ flux\ (W m^{-2})$',ylabel='height (m)')
out=ax[1].set(title=r'$\theta_l$ flux (K m/s)',ylabel='height (m)')



### calculate the saturated and unsaturated thetav jumps at the inversion

(unnumbered del Gesso equations below equation 14)

In [ ]:
dth=steady_state.deltheta
dqt=steady_state.delqv
thetal_m=steady_state.theta
qt_m=steady_state.qv
h=steady_state.h
press=tf.find_press(steady_state.h)   #kPa
thetal_ft = steady_state.theta + dth
qt_ft = steady_state.qv + dqt

Ad,Bd,issat = tf.calc_ABcoeffs(thetal_ft,qt_ft,press)
Aw,Bw,issat = tf.calc_ABcoeffs(thetal_m,qt_m,press)
invert= tf.t_uos_thetal(thetal_m,qt_m,press)
T_0 = invert.temp
lv=tf.L_t(invert.temp)
Cl =  (Ad*lv/tc.CPD - T_0/tc.EPS)
del_thv_dry = Ad * dth + Bd * dqt
del_thv_sat = Aw * dth + Bw * dqt
print('del_thv_dry',del_thv_dry)
print('del_thv_sat',del_thv_sat)

###  calculate the thetav inversion jump accounting for evaporative cooling

(see Stevens 2002 equations 16-18)

In [ ]:
ql_max = invert.ql
print('ql_max at cloud top (kg/kg)',ql_max)
Cl =  (Ad*lv/tc.CPD - T_0/tc.EPS)
Del_thv = del_thv_dry - Cl * ql_max
print('Del_thv',Del_thv)

### calculate the buoyancy integral terms

Gesso equations 13-14

In [ ]:
zi = steady_state.h
lcl_press=tf.LCL_thetal(thetal_m,qt_m)
zb=tf.find_height(lcl_press)

T1 = zb/zi
T2 = 0.5 * zb**2 / zi**2
T3 = (zi-zb)/zi
T4 = 0.5 * (zi**2 - zb**2) / zi**2

wtl_0=steady_state.T_flux_0
wqt_0=steady_state.q_flux_0
Del_F = steady_state.radcool/tc.CPD

term1 = wtl_0 * (Ad * (T1-T2) + Aw * (T3-T4))
term2 = wqt_0 * (Bd * (T1-T2) + Bw * (T3-T4))
term3 = Del_F * (Ad * T2      + Aw * T4)

Theta_NE = term1 + term2 + term3
print('Gesso Θne',Theta_NE)

Find wstar using Gesso equations 8 and 12

In [ ]:
wstar=(2.5*9.8/T_0*zi*Theta_NE)**(1/3.)
print('wstar without entrainment: {:5.3f} (m/s)'.format(wstar))

### Calculate the densest mixture for this inversion jump

Stevens equation 20

In [ ]:
chi_star = Cl * ql_max / (del_thv_dry - del_thv_sat)
print('just saturated at with {:5.2f}% environmental air'.format(chi_star*100.))

### Calculate the Nicholls and Turton average evaporative cooling effect $\Delta m / \Delta \theta_v$

Stevens eq. 30 and de Roode lecture 2  slide 12 -- note that $\Delta m/\Delta \theta_v = 1$ for no evaporation

In [ ]:
Del_m = del_thv_dry + chi_star * (2. - chi_star) * (del_thv_sat - del_thv_dry)
print('Δm/Δθv = {:5.2f}'.format(Del_m/Del_thv))

### Calculate the weakened Nicholls and Turton inversion buoyancy jump

de Roode lecture 2 slide 15 and Stevens eqn 29 with a2=15 following Bretherton

In [ ]:
a2=15.
Del_thv_NT = Del_thv / (1. + a2 * (1. - Del_m/Del_thv))
print('original Del_thv={:5.2f} (K), Del_thv_NT = {:5.2f} (K)'.format(Del_thv,Del_thv_NT))

### Calculate the final entrainment rate according to de Roode lecture 2 slide 15

In [ ]:
A_NT = 0.2
fac_NT = 2.5

term4 = Del_thv_NT
term5 = A_NT * fac_NT * (T2 * del_thv_dry + T4 * del_thv_sat)
denominator = term4 + term5

we = A_NT * fac_NT * Theta_NE / denominator
print('radiative we is {:5.3f} cm/s and NT we is {:5.3f} cm/s'.format(steady_state.went*100.,we*100.))